In [1]:
#importowanie bibliotek
import pandas as pd
import nltk
import string
from pathlib import Path
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [2]:
#pobranie potrzebny dany ze zbioru biblioteki nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

In [3]:
#zainicjowanie nowego obiektu PorterStemmer()
ps = PorterStemmer()

In [4]:
def cleanup():  
    # Usuwanie kolumny FILE_NAME
    spam_df.drop(columns="FILE_NAME", inplace=True)
    #usunięcie wierszy zduplikowanych
    spam_df.drop_duplicates(inplace=True)

In [5]:
#funkcja odpowiedzialna za czyszczenie tekstu mailu z pliku danych
def transform_text(text):
    text = text.lower() #sprowadzenie znaku do małych liter
    text = nltk.word_tokenize(text) #tokenizacja

    y = []
    for i in text:
        if i.isalnum():
            y.append(i) #sprawdzenie czy znak jest alfanumeryczny
    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i) #usunięcie wyrazów znajdujących się w stopwords i znaków interpunkcyjnych

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i)) #stemming słowa
    return " ".join(y)

In [6]:
#Sprawdzenie i ewentualne odczytanie pliku zawierającego już oczyszczony tekst w celu przyspieszenia pracy programu

my_file = Path("spam_NLP_cleaned.csv")

if my_file.is_file():
    spam_df = pd.read_csv(my_file)
else:
    #wczytanie pliku z listą danych
    spam_df = pd.read_csv("spam_NLP.csv")
    cleanup()
    spam_df["MESSAGE_COV"] = spam_df["MESSAGE"].apply(transform_text)
    spam_df.to_csv(my_file)

In [7]:
#wektoryzacja tekstu
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(spam_df["MESSAGE_COV"]).toarray()
y = spam_df["CATEGORY"].values

In [8]:
# Podzielenie danych na Train/Test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
# utworzenie modelu na podstawie Wielomianowego naiwnego klasyfikatora Bayesa
model = MultinomialNB()
model.fit(x_train, y_train)
#sprawdzenie i ocena danych testowych
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

confusion = confusion_matrix(y_test, y_pred)

#utworzenie ciągu znaków z ocenami modelu
Scores_String = "Accuracy: " + str(round(accuracy, 4))
Scores_String += "\nPrecision: " + str(round(precision, 4))
Scores_String += "\nRecall: " + str(round(recall, 4))
Scores_String += "\nF1 Score: " + str(round(f1, 4))

Scores_String += "\n\nConfusion Matrix:\n" + str(confusion)

In [10]:
#Wczytanie pliku z w wygenerowanymi danymi testowymi
test_data = pd.read_csv("TEST_DATA_spam.csv",encoding="ANSI") 